# Post Call Analytics

Welcome to this training module on post-call analytics use cases using Amazon SageMaker JumpStart. 

As businesses continue to interact with customers through various channels, it becomes increasingly important to analyze these interactions to gain insights into customer behavior and preferences. Post-call analytics is one such method that involves analyzing customer interactions after the call has ended. The use of large language models can greatly enhance the effectiveness of post-call analytics by enabling more accurate sentiment analysis, identifying specific customer needs and preferences, and improving overall customer experience. 

In this sample notebook, we will explore following topics to demonstrate the various benefits of using Bedrock for post-call analytics and businesses gain a competitive edge in the modern marketplace.

- Choice of LLM models in Amazon SageMaker JumpStart
- One model handling multiple PCA tasks
- Handling long call transcripts

## Step 0. Install packages

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [2]:
install_needed = True  # should only be True once

In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U langchain

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


## Step 1. Prepare Large Language Model (LLM)

In [4]:
import boto3
from termcolor import colored
from sagemaker.session import Session

from lib_ko import get_payload
from langchain.llms import AmazonAPIGateway
from lib_en import Llama2ContentHandlerAmazonAPIGateway, FalconContentHandlerEndpoint, FalconContentHandlerAmazonAPIGateway

In [5]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

In [6]:
MODEL_NAME = "KULLM-12-8B"

In [7]:
RESTAPI_ID = "6bk4r5mo4f"
URL = f'https://{RESTAPI_ID}.execute-api.{aws_region}.amazonaws.com/api/'.replace('"','')
LLM_INFO = {
    "KULLM-12-8B": f"{URL}llm/kkulm_12_8b",
}
LLM_URL = LLM_INFO[MODEL_NAME]
HEADERS = {    
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

print (f'MODEL_NAME: {MODEL_NAME}\nLLM_URL: {LLM_URL}')

MODEL_NAME: KULLM-12-8B
LLM_URL: https://6bk4r5mo4f.execute-api.us-east-1.amazonaws.com/api/llm/kkulm_12_8b


In [8]:
llm = AmazonAPIGateway(api_url=LLM_URL, headers=HEADERS)

## Step 2. Load transcript files

In [26]:
transcript_files = [
    "./call_transcripts/negative-refund-ko.txt", 
    "./call_transcripts/neutral-short-ko.txt",
    "./call_transcripts/positive-partial-refund-ko.txt",
    "./call_transcripts/aws-ko-short.txt",
    "./call_transcripts/aws-ko.txt"
]
transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())

In [27]:
for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

transcript #1: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 



transcript #2: timestamp: 2023-01-28 08:26:49.219717

고객: 안녕하세요, 제 계정의 잔액을 확인하고 싶습니다.

상담원: 물론이죠! 계정에 연결된 계좌 번호나 전화번호를 알려주실 수 있나요?

고객: 네: 제 전화번호는 (123) 456-7890입니다.

상담원: 네, 감사합니다. 계정을 불러올게요. 현재 잔액이 $567.89인 것 같습니다.

고객: 네, 좋아요. 감사합니다.

상담원: 천만에요! 오늘 또 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 감사합니다.

상담원: 문제 없습니다. 전화해 주셔서



transcript #3: timestamp: 2022-12-28 08:26:49.219717

상담원: 소매업체]에 전화해 주셔서 감사합니다. 제 이름은 [상담원 이름]입니다. 오늘은 무엇을 도와드릴까요?

고객: 안녕하세요, 주문 상태를 확인하고 싶어서요. 오늘 도착하기로 되어 있었는데 아직 받지 못했습니다.

상담원: 유감입니다. 주문 번호를 알려주시겠습니까?

고객: 네, 123456입니다.

상담원: 네: 감사합니다. 제가 확인해 보겠습니다. 창고에서 예기치 않은 문제가 발생하여 주문이 며칠 지연된 것 같습니다. 불편을 드려 죄송합니다.

고객: 괜



transcript #4: AWS란 무엇인가요? AWS 또는 Amazon We

## Step 3. Post Call Analysis

In [28]:
from langchain import PromptTemplate

### Step 3.1. Prompt Template
In this notebook, we'll be performing four different analyses(**Summary, Sentiment, Intent and Resolution**), and we'll need a template for each one. 

* Summary template

In [29]:
summary_template = """
아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
다음 글을 알기 쉽게 요약해 주세요.

### 입력: {transcript}


### 응답:
"""

### Step 3.2. Analysis

In [30]:
def analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    analysis_prompt = prompt.format(transcript=transcript)
    llm.model_kwargs = params
    
    print (colored(analysis_prompt, 'green'))
    response = llm(analysis_prompt)
    
    return response

In [31]:
PARAMS = {
    "KULLM-12-8B": {
        "do_sample": False,
        "max_new_tokens": 256,
        "temperature": 0.0,
        "top_p": 0.9,
        "return_full_text": False,
        "repetition_penalty": 1.2,
        "presence_penalty": None,
        "eos_token_id": 2,
    },
}

* Summary analysis

In [32]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[1],
    params=PARAMS[MODEL_NAME],
    template=summary_template
)

print (res)


아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
다음 글을 알기 쉽게 요약해 주세요.

### 입력: timestamp: 2023-01-28 08:26:49.219717

고객: 안녕하세요, 제 계정의 잔액을 확인하고 싶습니다.

상담원: 물론이죠! 계정에 연결된 계좌 번호나 전화번호를 알려주실 수 있나요?

고객: 네: 제 전화번호는 (123) 456-7890입니다.

상담원: 네, 감사합니다. 계정을 불러올게요. 현재 잔액이 $567.89인 것 같습니다.

고객: 네, 좋아요. 감사합니다.

상담원: 천만에요! 오늘 또 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 감사합니다.

상담원: 문제 없습니다. 전화해 주셔서 감사합니다. 좋은 하루 되세요!

고객: 저도요. 안녕히 가세요.

상담원: 안녕히 가세요!


### 응답:

이 고객의 계정 잔액을 확인하려고 합니다. 고객님께서 알려주시면 계정과 연결되어 있는 은행 또는 전화번호로 계정을 불러오겠습니다. 고객님의 전화번호가 1234567890이면 계정을 불러올 수 있고, 이 경우 현재 잔액은 56709달러 87센트입니다.
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 3.61 s


## Handling long call transcripts
We'll cover how to handle long transcripts that exceed the limits of the LLM. 

In [33]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [34]:
def long_call_analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning
    
    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")
        summary_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True) # map_reduce, refine

        transcript = summary_chain.run(docs)

    analysis_prompt = prompt.format(transcript=transcript) 
    analysis = llm(analysis_prompt)

    return analysis

In [35]:
PARAMS = {
    "KULLM-12-8B": {
        "do_sample": False,
        "max_new_tokens": 512,
        "temperature": 0.1,
        "top_p": 0.9,
        "return_full_text": False,
        "repetition_penalty": 1.2,
        "presence_penalty": None,
        "eos_token_id": 2,
    },
}

In [36]:
%%time

res = long_call_analysis(
    llm=llm,
    transcript=transcripts[3],
    params=PARAMS[MODEL_NAME],
    template=summary_template
)

print (res)

Token indices sequence length is longer than the specified maximum sequence length for this model (2950 > 1024). Running this sequence through the model will result in indexing errors


Now we have 4 documents and the first one has 1001 tokens


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"AWS란 무엇인가요? AWS 또는 Amazon Web Services는 공용 인터넷을 통해 액세스할 수 있는 다양한 컴퓨팅 서비스를 제공하는 클라우드 서비스 제공업체입니다.

AWS 및 기타 퍼블릭 클라우드 공급업체(예: Google Cloud Platform(GCP) 및 Microsoft Azure)는 하드웨어와 인프라를 관리 및 유지 관리하여 조직과 개인이 현장에서 리소스를 구매하고 실행하는 데 드는 비용과 복잡성을 덜어줍니다. 이러한 리소스는 무료 또는 사용량 기반 유료로 액세스할 수 있습니다.

AWS를 더 잘 이해하려면 AWS가 얼마나 방대한지 이해하는 것이 도움이 될 수 있습니다. 부정할 수 없는 사실은 AWS가 엄청나게 크다는 것입니다. 얼마나 큰 규모일까요?

인터넷에서 방문하는 사이트 세 곳 중 한 곳은 AWS 서비스를 사용합니다. 
2019년 아마존 웹 서비스는 350억 달러 이상의 매출을 올렸습니다. AWS가 단독 기업이었다면 포춘지 선정 글로벌 500대 기업에서 359위를 차지할 수 있는 규모입니다.
이제 AWS에 대한 10,000피트 높이의 개요를 살펴보겠습니다. 자세히 살펴보겠습니다!"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point:  "클라우드 컴퓨팅이란 무엇인가?"